In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import numpy as np
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64)



In [6]:
DEFAULT_WIDTH = 30
DEFAULT_DEPTH = 7

class PINN_f(nn.Module):
    def __init__(self, width=DEFAULT_WIDTH, depth=DEFAULT_DEPTH):
        super(PINN_f, self).__init__()
        self.width = width
        if depth < 2:
          raise ValueError(f"depth must be at least 2, got {depth}")
        self.depth = depth
        self.act = nn.Tanh()

        # layer definitions
        self.FC_dict = nn.ModuleDict()
        self.FC_dict[f"layer 1"] = nn.Linear(2, self.width)
        for i in range(2, depth):
          self.FC_dict[f"layer {i}"] = nn.Linear(self.width, self.width)
        self.FC_dict[f"layer {depth}"] = nn.Linear(self.width, 1)

    def forward(self, x):
      for key, layer in self.FC_dict.items():
        x = layer(x)
        if key != f"layer {self.depth}":
          x = self.act(x)
      return x


In [10]:
torch.manual_seed(0)
np.random.seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_phi = PINN_f().to(device)
model_psi = PINN_f().to(device)
model_w = PINN_f().to(device)
model_u1 = PINN_f().to(device)
model_u2 = PINN_f().to(device)

print(model_u1)
d_INIT = 3
l = torch.tensor([d_INIT,], dtype=torch.float64, device=device, requires_grad=True)
# define an optimizer
params = list(model_psi.parameters()) + list(model_w.parameters())+ list(model_phi.parameters()) + list(model_u1.parameters())+list(model_u2.parameters())


nn_optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0)
#nn_scheduler = torch.optim.lr_scheduler.StepLR(nn_optimizer, step_size=1000, gamma=0.5)
l_optimizer = torch.optim.Adam([l], lr=0.001, weight_decay=0) #l learning rate can be even better
#l_scheduler = torch.optim.lr_scheduler.StepLR(l_optimizer, step_size=1000, gamma=0.5)

PINN_f(
  (act): Tanh()
  (FC_dict): ModuleDict(
    (layer 1): Linear(in_features=2, out_features=30, bias=True)
    (layer 2): Linear(in_features=30, out_features=30, bias=True)
    (layer 3): Linear(in_features=30, out_features=30, bias=True)
    (layer 4): Linear(in_features=30, out_features=30, bias=True)
    (layer 5): Linear(in_features=30, out_features=30, bias=True)
    (layer 6): Linear(in_features=30, out_features=30, bias=True)
    (layer 7): Linear(in_features=30, out_features=1, bias=True)
  )
)


In [32]:
# to verify that we have a small loss using approx solutions
l=10
alpha=1/(1+l)
c_norm=24/13/np.pi
batchsize = 5000 #interior
batchsize1 = 500 #boundary, smoothness? (close to the origin)
for ep in range(100000):
    nn_optimizer.zero_grad()

    #l_optimizer.zero_grad()

    ### equation loss
    # sample points uniformly in the transformed coordinates [-30,30]x[0,30]
    c = 30
    x1 = c*torch.rand(batchsize, 1, device=device)# (batch, 1)
    x1.requires_grad=True
    x2 = c*torch.rand(batchsize, 1, device=device) # (batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    z1=torch.sinh(x1)
    z2=torch.sinh(x2)
    gamma=torch.atan(z2/z1)
    R=(z1**2+z2**2)**(alpha/2)
    r=(z1**2+z2**2)**(1/2)
    w=-alpha/c_norm*torch.cos(gamma)**alpha*3*R/(1+R)**2
    phi=-alpha/c_norm*torch.cos(gamma)**alpha*6*R/(1+R)**3
    psi=0*r
    u1=-3*z1/(1+R)#z1
    u2=3*z2/(1+R)#z2
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    
    loss_i_1=torch.norm(f_1)**2/batchsize
    loss_i_2=torch.norm(f_2)**2/batchsize
    loss_i_3=torch.norm(f_3)**2/batchsize
    loss_i_4=torch.norm(f_4)**2/batchsize
    loss_i_5=torch.norm(f_5)**2/batchsize
    loss_i_6=torch.norm(f_6)**2/batchsize
    
    loss_i = (loss_i_1+loss_i_2+loss_i_3+loss_i_4+loss_i_5+loss_i_6)/N_e
    
    ###boundary loss
    # sample points
    y1 = c*(torch.rand(batchsize1, 1, device=device))# (batch1, 1)
    y1.requires_grad=True
    y2 = c*torch.rand(batchsize1, 1, device=device) # (batch1, 1)
    y2.requires_grad=True
    y0 = torch.zeros(batchsize1,1, device=device,requires_grad=True)
    yd = c*torch.ones(batchsize1,1, device=device)
    yd.requires_grad=True
    
    #boundary 1
    input1_b1 = torch.cat([y1,y0], dim=-1) # (batch1, 2)
    input2_b1 = torch.cat([-y1,y0], dim=-1) # (batch1, 2)
    
    z1=torch.sinh(y1)
    z2=torch.sinh(y0)
    gamma=torch.atan(z2/z1)
    R=(z1**2+z2**2)**(alpha/2)
    u2_b1=3*z2/(1+R)#z2
    b_1 = u2_b1
    loss_b_1 = torch.norm(b_1)**2/batchsize1
    
    
    #boundary 3,5,7
    input1_b3 = torch.cat([yd,y2], dim=-1) # (batch1, 2)
    input2_b3 = torch.cat([-yd,y2], dim=-1) # (batch1, 2)
    
    z1=torch.sinh(yd)
    z2=torch.sinh(y2)
    gamma=torch.atan(z2/z1)
    R=(z1**2+z2**2)**(alpha/2)
    phi_b3=-alpha/c_norm*torch.cos(gamma)**alpha*6*R/(1+R)**3
    psi_b3=0*R
    u1_b3=-3*z1/(1+R)#z1
    u2_b3=3*z2/(1+R)#z2
    u1_1_b3 = grad(u1_b3.sum(), yd, create_graph=True)[0]
    u1_2_b3 = grad(u1_b3.sum(), y2, create_graph=True)[0]
    u2_1_b3 = grad(u2_b3.sum(), yd, create_graph=True)[0]
    u2_2_b3 = grad(u2_b3.sum(), y2, create_graph=True)[0]
    b_31=u1_1_b3/torch.cosh(yd)
    b_33=u1_2_b3/torch.cosh(y2)
    b_34=u2_1_b3/torch.cosh(yd)
    b_32=u2_2_b3/torch.cosh(y2)
    b_5=phi_b3
    b_7=psi_b3
    loss_b_3 = (torch.norm(b_31)**2+torch.norm(b_32)**2+torch.norm(b_33)**2+torch.norm(b_34)**2+torch.norm(b_5)**2+torch.norm(b_7)**2)/batchsize1
    
    #boundary 4,6,8
    input1_b4 = torch.cat([y1,yd], dim=-1) # (batch1, 2)
    input2_b4 = torch.cat([-y1,yd], dim=-1) # (batch1, 2)
    
    z1=torch.sinh(y1)
    z2=torch.sinh(yd)
    gamma=torch.atan(z2/z1)
    R=(z1**2+z2**2)**(alpha/2)
    phi_b4=-alpha/c_norm*torch.cos(gamma)**alpha*6*R/(1+R)**3
    psi_b4=0*R
    u1_b4=-3*z1/(1+R)#z1
    u2_b4=3*z2/(1+R)#z2
    u1_1_b4 = grad(u1_b4.sum(), y1, create_graph=True)[0]
    u1_2_b4 = grad(u1_b4.sum(), yd, create_graph=True)[0]
    u2_1_b4 = grad(u2_b4.sum(), y1, create_graph=True)[0]
    u2_2_b4 = grad(u2_b4.sum(), yd, create_graph=True)[0]
    b_41=u1_1_b4/torch.cosh(y1)
    b_43=u1_2_b4/torch.cosh(yd)
    b_44=u2_1_b4/torch.cosh(y1)
    b_42=u2_2_b4/torch.cosh(yd)
    b_6=phi_b4
    b_8=psi_b4
    loss_b_4 = (torch.norm(b_41)**2+torch.norm(b_42)**2+torch.norm(b_43)**2+torch.norm(b_44)**2+torch.norm(b_6)**2+torch.norm(b_8)**2)/batchsize1
    
    #collect the residue
    N_b=8
    loss_b = (loss_b_1+loss_b_3+loss_b_4)/N_b
    
    ### smoothness loss
    # sample points uniformly in the transformed coordinates [-1,1]x[0,1]
   
    x1 = c*torch.rand(batchsize1, 1, device=device)# (batch, 1)
    x1.requires_grad=True
    x2 = c*torch.rand(batchsize1, 1, device=device) # (batch, 1)
    x2.requires_grad=True
    input1 = torch.cat([x1,x2], dim=-1) # (batch, 2)
    input2 = torch.cat([-x1,x2], dim=-1) # (batch, 2)
    z1=torch.sinh(x1)
    z2=torch.sinh(x2)
    gamma=torch.atan(z2/z1)
    R=(z1**2+z2**2)**(alpha/2)
    r=(z1**2+z2**2)**(1/2)
    w=-alpha/c_norm*torch.cos(gamma)**alpha*3*R/(1+R)**2
    phi=-alpha/c_norm*torch.cos(gamma)**alpha*6*R/(1+R)**3
    psi=0*r
    u1=-3*z1/(1+R)#z1
    u2=3*z2/(1+R)#z2
    
    
    #compute the derivatives (only first orders)
    w_1 = grad(w.sum(), x1, create_graph=True)[0]
    w_2 = grad(w.sum(), x2, create_graph=True)[0]
    phi_1 = grad(phi.sum(), x1, create_graph=True)[0]
    phi_2 = grad(phi.sum(), x2, create_graph=True)[0]
    psi_1 = grad(psi.sum(), x1, create_graph=True)[0]
    psi_2 = grad(psi.sum(), x2, create_graph=True)[0]
    u1_1 = grad(u1.sum(), x1, create_graph=True)[0]
    u1_2 = grad(u1.sum(), x2, create_graph=True)[0]
    u2_1 = grad(u2.sum(), x1, create_graph=True)[0]
    u2_2 = grad(u2.sum(), x2, create_graph=True)[0]
    
    #compute the equation residue
    N_e = 6
    f_1 = w+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*w_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*w_2-phi
    f_2 = (2+u1_1/torch.cosh(x1))*phi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*phi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*phi_2+u2_1/torch.cosh(x1)*psi
    f_3 = (2+u2_2/torch.cosh(x2))*psi+((1+l)*torch.sinh(x1)+u1)/torch.cosh(x1)*psi_1+((1+l)*torch.sinh(x2)+u2)/torch.cosh(x2)*psi_2+u1_2/torch.cosh(x2)*phi
    f_4 = u1_1/torch.cosh(x1)+u2_2/torch.cosh(x2)
    f_5 = w-(u1_2/torch.cosh(x2)-u2_1/torch.cosh(x1))
    f_6 = phi_2/torch.cosh(x2)-psi_1/torch.cosh(x1)
    

    
    #compute the residue derivatives
    F_1_1=grad(f_1.sum(), x1, create_graph=True)[0]
    F_1_2=grad(f_1.sum(), x2, create_graph=True)[0]
    F_2_1=grad(f_2.sum(), x1, create_graph=True)[0]
    F_2_2=grad(f_2.sum(), x2, create_graph=True)[0]
    F_3_1=grad(f_3.sum(), x1, create_graph=True)[0]
    F_3_2=grad(f_3.sum(), x2, create_graph=True)[0]
    F_4_1=grad(f_4.sum(), x1, create_graph=True)[0]
    F_4_2=grad(f_4.sum(), x2, create_graph=True)[0]
    F_5_1=grad(f_5.sum(), x1, create_graph=True)[0]
    F_5_2=grad(f_5.sum(), x2, create_graph=True)[0]
    F_6_1=grad(f_6.sum(), x1, create_graph=True)[0]
    F_6_2=grad(f_6.sum(), x2, create_graph=True)[0]
    

    loss_s_1=(torch.norm(F_1_1)**2+torch.norm(F_1_2)**2)/batchsize1
    loss_s_2=(torch.norm(F_2_1)**2+torch.norm(F_2_2)**2)/batchsize1
    loss_s_3=(torch.norm(F_3_1)**2+torch.norm(F_3_2)**2)/batchsize1
    loss_s_4=(torch.norm(F_4_1)**2+torch.norm(F_4_2)**2)/batchsize1
    loss_s_5=(torch.norm(F_5_1)**2+torch.norm(F_5_2)**2)/batchsize1
    loss_s_6=(torch.norm(F_6_1)**2+torch.norm(F_6_2)**2)/batchsize1
    

    loss_s = (loss_s_1+loss_s_2+loss_s_3+loss_s_4+loss_s_5+loss_s_6)/N_e
    
    #the total loss
    gamma=0.1
    loss =gamma*(loss_i+loss_s)+1*loss_b

    # update the model
    loss.backward()
    nn_optimizer.step()
    #nn_scheduler.step()

    #l_optimizer.step()
    #l_scheduler.step()

    # plot
    if ep % 1000 == 0:
        print("Epoch is {},  function loss is {}, boundary loss is {}, smoothness loss is {}, overall loss is {} ".format(ep, loss_i.item(),loss_b.item(),loss_s.item(), loss.item()))
        #print(ep, 'loss' +{loss_i.item()}, loss_b.item(),loss1.item(),loss2.item(), loss.item())
        print(l)
        # uniformly sample from [r,z] 
       


Epoch is 0,  function loss is 0.0008530763867466242, boundary loss is 0.017512156595954995, smoothness loss is 7.897908582589887e-05, overall loss is 0.017605362143212247 
10


KeyboardInterrupt: 

In [27]:
b_32

tensor([[1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2838],
        [1.2839],
        [1.2839],
        [1.2838],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2788],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2834],
        [1.2839],
        [1.2820],
        [1.2839],
        [1.2786],
        [1.2827],
        [1.2785],
        [1.2839],
        [1.2821],
        [1.2835],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1.2839],
        [1